In [1]:
%matplotlib inline
from pylab import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from networkx.drawing.nx_agraph import write_dot, graphviz_layout
from pyomo.environ import ConcreteModel, SolverFactory, TerminationCondition, \
    value, Var, Constraint, Expression, Objective, TransformationFactory, units as pyunits
from pyomo.network import Arc, SequentialDecomposition
from idaes.core.util.model_statistics import degrees_of_freedom

from split_test2 import Separator1

from mixer_example import Mixer1

### TODO: CREATE INTRO

##### Import WaterTAP3 Package

In [2]:
import watertap as wt

##### Step 1: Look up unit process library. Returns a List.

##### Step 3: Set up IDAES flowsheet for watertap. This selects a property package for you (see water_props.py). The property package could be changed in the future for performing heat/energy and other balances.

In [3]:
m = wt.watertap_setup(dynamic = False)

##### Step 3: Add a water source and define inlet flow.

In [4]:
m = wt.case_study_trains.get_case_study(name = 'carlsbad', flow = 4.5833, m = m)

/Users/amiara/NAWI/WaterTap/NAWI-WaterTAP3/IDAES-WaterTAP3_v2/ml_regression.py:149: RuntimeWarning: divide by zero encountered in power
  return a*np.power(x, b)
/Users/amiara/NAWI/WaterTap/NAWI-WaterTAP3/IDAES-WaterTAP3_v2/ml_regression.py:149: RuntimeWarning: divide by zero encountered in power
  return a*np.power(x, b)
/Users/amiara/NAWI/WaterTap/NAWI-WaterTAP3/IDAES-WaterTAP3_v2/ml_regression.py:149: RuntimeWarning: divide by zero encountered in power
  return a*np.power(x, b)
/Users/amiara/NAWI/WaterTap/NAWI-WaterTAP3/IDAES-WaterTAP3_v2/ml_regression.py:149: RuntimeWarning: divide by zero encountered in power
  return a*np.power(x, b)
/Users/amiara/NAWI/WaterTap/NAWI-WaterTAP3/IDAES-WaterTAP3_v2/ml_regression.py:149: RuntimeWarning: divide by zero encountered in power
  return a*np.power(x, b)
/Users/amiara/NAWI/WaterTap/NAWI-WaterTAP3/IDAES-WaterTAP3_v2/ml_regression.py:149: RuntimeWarning: divide by zero encountered in power
  return a*np.power(x, b)
/Users/amiara/NAWI/WaterTap/

In [5]:
wt.display.show_train2(model_name=m)

In [6]:
##### Step 5:  Run model and display results. The resutls display will be put into easier format to read for user soon.

In [7]:
result = wt.run_water_tap(m)

Ipopt 3.12.13: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.13, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      755
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      133

Total number of variables............................:      283
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        1
                     variables with only upper bounds:        0
Tot

In [49]:
unit_process_names = [
    "swoi", 
    "coag_floc",
    "tri_media_filtration",
    "SAA",
    "SBA",
    "cf",
    "ro",
    "lime",
    "co2",
    "chlor",
    "ammonia",
    "TWS_24_hr",
    "muni"
]


cost_variables = [
    "total_up_cost",
    "fixed_cap_inv_unadjusted",
    "fixed_cap_inv",
    "total_fixed_op_cost",
    "total_cap_investment",
    "land_cost",
    "working_cap",
    "catalysts_chemicals",#
    #"electricity",##
    "cap_replacement_parts", #
    "salaries",
    "benefits",
    "maintenance",
    "lab",
    "insurance_taxes",
    "base_employee_salary_cost",
    
    "cat_and_chem_cost",##
    "labor_and_other_fixed",#
    "consumer_price_index",#
    
    "electricity_cost", ##
    "other_var_cost"   ## 
]

In [50]:
def get_results_table(model, unit_process_names):
    data = {} 

    for variable in cost_variables:
        cost_list = []
        for node in unit_process_names:
            # call m.fs.'node'.costing.'variable'() for each node and variable
            costing_attr = getattr(m.fs, node).costing
            cost_method = getattr(costing_attr, variable) #m.fs.swoi.costing.total_up_cost
            
            #Todo: add unit (e.g. million $ or m3/s etc.)
            
            if type(cost_method) == int:
                cost_method = np.float64(cost_method)
                
            if type(cost_method) == np.float64:
                cost_value = cost_method
            else:
                cost_value = cost_method()
            # append ^ to a list
            cost_list.append(cost_value)            
            
        # assign 'variable': List['node' values] in a data dictionary
        data[variable] = cost_list
    
    
    df = pd.DataFrame(data = data, index = unit_process_names, columns = cost_variables)
    
    
    
    #print(df)
    return df
    

In [51]:
get_results_table(m, unit_process_names).to_csv("excel_comparison.csv")

In [52]:
get_results_table(m, unit_process_names).head()

,total_up_cost,fixed_cap_inv_unadjusted,fixed_cap_inv,total_fixed_op_cost,total_cap_investment,land_cost,working_cap,catalysts_chemicals,cap_replacement_parts,salaries,benefits,maintenance,lab,insurance_taxes,base_employee_salary_cost,cat_and_chem_cost,labor_and_other_fixed,consumer_price_index,electricity_cost,other_var_cost
swoi,23.366026,20.308784,20.980305,1.305235,22.060791,0.031470,1.049015,1.045314,1.033066,0.543417,0.489075,0.167842,0.062941,0.041961,0.020309,0.000000,1.032158,1.034582,1.984638,-1.984638
coag_floc,14.211373,11.765810,11.765810,0.732488,12.371749,0.017649,0.588291,1.000000,1.000000,0.305017,0.274515,0.094126,0.035297,0.023532,0.011766,1.008727,1.000000,1.000000,0.098409,0.000000
tri_media_filtration,42.354232,11.748013,12.751178,24.703293,13.407863,0.019127,0.637559,1.094885,1.085390,12.914488,11.623039,0.102009,0.038254,0.025502,0.011748,0.000000,1.144289,1.104141,4.243076,0.000000
SAA,0.716879,0.351380,0.418187,0.027077,0.439723,0.000627,0.020909,1.449793,1.190128,0.011390,0.010251,0.003345,0.001255,0.000836,0.000351,0.250068,1.255215,1.260734,0.000011,0.000000
SBA,1.217590,0.351816,0.418706,0.027110,0.440270,0.000628,0.020935,1.449793,1.190128,0.011404,0.010263,0.003350,0.001256,0.000837,0.000352,0.750197,1.255215,1.260734,0.000013,0.000000


In [10]:
variables = cost_variables + ["inlet_flow", "outlet_flow", "waste_flow"]

In [11]:
def get_results_table(model, unit_process_names, scenario = None, case_study = None):
    # could make a dictionary if betteR?
    value_list = [] 
    up_list = []
    scenario_list = []
    unit_list = []
    variable_list = []
    case_study_list = []
    
    for node in unit_process_names:
        
        for variable in variables:
            
            if variable == "inlet_flow":
                value_list.append(getattr(m.fs, node).inlet.flow_vol.extract_values()[0])
                variable_list.append(variable)
                unit_list.append("m3/s")
            elif variable == "outlet_flow":
                value_list.append(getattr(m.fs, node).outlet.flow_vol.extract_values()[0])
                variable_list.append(variable)
                unit_list.append("m3/s")
            elif variable == "waste_flow":
                value_list.append(getattr(m.fs, node).waste.flow_vol.extract_values()[0])
                variable_list.append(variable)
                unit_list.append("m3/s")
            else:
                # call m.fs.'node'.costing.'variable'() for each node and variable
                costing_attr = getattr(m.fs, node).costing
                cost_method = getattr(costing_attr, variable) #m.fs.swoi.costing.total_up_cost            
            
                if type(cost_method) == int:
                    cost_method = np.float64(cost_method)

                if type(cost_method) == np.float64:
                    cost_value = cost_method
                else:
                    cost_value = cost_method()
                # append ^ to a list
                value_list.append(cost_value)            
                variable_list.append(variable)
                unit_list.append("$MM")        
        
            up_list.append(node)
            scenario_list.append(scenario)
            case_study_list.append(case_study)
    
    df = pd.DataFrame()
    df["Case_Study"] = case_study_list
    df["Scenario"] = scenario_list
    df["Unit_Process"] = up_list
    df["Variable"] = variable_list
    df["Unit"] = unit_list
    df["Value"] = value_list
    
    #print(df)
    return df

In [12]:
t1 = get_results_table(m, unit_process_names, scenario = "Baseline", case_study = "Carlsbad")

In [13]:
t1.head()

,Case_Study,Scenario,Unit_Process,Variable,Unit,Value
0,Carlsbad,Baseline,swoi,total_up_cost,$MM,23.366026
1,Carlsbad,Baseline,swoi,fixed_cap_inv_unadjusted,$MM,20.308784
2,Carlsbad,Baseline,swoi,fixed_cap_inv,$MM,20.980305
3,Carlsbad,Baseline,swoi,total_fixed_op_cost,$MM,1.305235
4,Carlsbad,Baseline,swoi,total_cap_investment,$MM,22.060791


In [14]:
t1.to_csv("results_v2.csv")

In [15]:
m.fs.tri_media_filtration.inlet.flow_vol.extract_values()[0]

5.064327456187778

In [16]:
m.fs.tri_media_filtration.costing.salaries()

12.914488212339975

In [17]:
m.fs.chlor.costing.catalysts_chemicals

1.0948846182937026

In [18]:
m.fs.chlor.costing.applied_cl2_dose * 3.78541178 * (0.34) * 1.0948846182937026 * 365 * 2.399478 * 22.824465227271


267605.01864422136

In [31]:
t1[t1.Unit_Process == "swoi"]

,Case_Study,Scenario,Unit_Process,Variable,Unit,Value
0,Carlsbad,Baseline,swoi,total_up_cost,$MM,23.366026
1,Carlsbad,Baseline,swoi,fixed_cap_inv_unadjusted,$MM,20.308784
2,Carlsbad,Baseline,swoi,fixed_cap_inv,$MM,20.980305
3,Carlsbad,Baseline,swoi,total_fixed_op_cost,$MM,1.305235
4,Carlsbad,Baseline,swoi,total_cap_investment,$MM,22.060791
5,Carlsbad,Baseline,swoi,land_cost,$MM,0.031470
6,Carlsbad,Baseline,swoi,working_cap,$MM,1.049015
7,Carlsbad,Baseline,swoi,catalysts_chemicals,$MM,1.045314
8,Carlsbad,Baseline,swoi,cap_replacement_parts,$MM,1.033066
9,Carlsbad,Baseline,swoi,salaries,$MM,0.543417


In [48]:
results_table = get_results_table(m, unit_process_name)

In [49]:
results_table.total_up_cost.sum()

462.8602092987865

In [50]:
results_table

,total_up_cost,fixed_cap_inv_unadjusted,fixed_cap_inv,total_fixed_op_cost,total_cap_investment,land_cost,working_cap,catalysts_chemicals,cap_replacement_parts,salaries,benefits,maintenance,lab,insurance_taxes,base_employee_salary_cost,cat_and_chem_cost,labor_and_other_fixed,consumer_price_index,electricity_cost,other_var_cost
swoi,23.366026,20.308784,20.980305,1.305235,22.060791,0.031470,1.049015,1.045314,1.033066,0.543417,0.489075,0.167842,0.062941,0.041961,0.020309,0.000000,1.032158,1.034582,1.984638,-1.984638
coag_floc,15.121692,11.765810,11.765810,0.732488,12.371749,0.017649,0.588291,1.000000,1.000000,0.305017,0.274515,0.094126,0.035297,0.023532,0.011766,1.008727,1.000000,1.000000,0.098409,0.910319
tri_media_filtration,14.283696,11.748013,12.751178,0.875833,13.407863,0.019127,0.637559,1.094885,1.085390,0.373720,0.336348,0.102009,0.038254,0.025502,0.011748,0.000000,1.144289,1.104141,4.243076,-4.243076
SAA,0.966936,0.351380,0.418187,0.027077,0.439723,0.000627,0.020909,1.449793,1.190128,0.011390,0.010251,0.003345,0.001255,0.000836,0.000351,0.250068,1.255215,1.260734,0.000011,0.250057
SBA,1.967774,0.351816,0.418706,0.027110,0.440270,0.000628,0.020935,1.449793,1.190128,0.011404,0.010263,0.003350,0.001256,0.000837,0.000352,0.750197,1.255215,1.260734,0.000013,0.750183
cf,13.380566,11.044252,11.987322,0.775897,12.604669,0.017981,0.599366,1.094885,1.085390,0.326348,0.293714,0.095899,0.035962,0.023975,0.011044,0.000000,1.144289,1.104141,0.000000,-0.000000
ro,296.404263,223.115886,265.536470,17.192665,279.211598,0.398305,13.276824,1.449793,1.190128,7.231943,6.508748,2.124292,0.796609,0.531073,0.223116,0.000000,1.255215,1.260734,33.747012,-33.747012
lime,59.544991,52.909213,52.909213,2.344555,55.634037,0.079364,2.645461,1.000000,1.000000,0.871966,0.784769,0.423274,0.158728,0.105818,0.052909,0.783200,1.000000,1.000000,0.101469,0.681730
co2,8.515704,7.646860,7.771228,0.344258,8.171446,0.011657,0.388561,1.022155,1.016264,0.128017,0.115215,0.062170,0.023314,0.015542,0.007647,0.000000,1.015824,1.016997,0.101468,-0.101468
chlor,10.362694,8.699168,9.441991,0.434440,9.928254,0.014163,0.472100,1.094885,1.085390,0.164050,0.147645,0.075536,0.028326,0.018884,0.008699,0.000000,1.144289,1.104141,0.101467,-0.101467


In [51]:
#results_table = pd.read_csv('./cost_results.csv',index_col=0)
results_table.index.names=['Unit_Process']
results_table = results_table.reset_index()
df = pd.melt(results_table, id_vars=['Unit_Process'], value_vars=results_table.columns.to_list()[1:])

In [52]:
df["Case_Study"] = "Carlsbad"
df["Unit"] = "Million $"
df["Scenario"] = "A"


In [53]:
df

,Unit_Process,variable,value,Case_Study,Unit
0,swoi,total_up_cost,23.366026,Carlsbad,Million $
1,coag_floc,total_up_cost,15.121692,Carlsbad,Million $
2,tri_media_filtration,total_up_cost,14.283696,Carlsbad,Million $
3,SAA,total_up_cost,0.966936,Carlsbad,Million $
4,SBA,total_up_cost,1.967774,Carlsbad,Million $
...,...,...,...,...,...
255,co2,other_var_cost,-0.101468,Carlsbad,Million $
256,chlor,other_var_cost,-0.101467,Carlsbad,Million $
257,ammonia,other_var_cost,-0.000012,Carlsbad,Million $
258,TWS_24_hr,other_var_cost,-0.000000,Carlsbad,Million $


In [60]:
m.fs.tri_media_filtration.inlet.flow_vol.display()

flow_vol_in : Volumetric flowrate of water into unit
    Size=1, Index=fs.time
    Key : Lower : Value             : Upper : Fixed : Stale : Domain
    0.0 :  None : 5.064327456187778 :  None : False : False :  Reals


In [64]:
m.fs.tri_media_filtration.inlet.flow_vol.extract_values()[0]

5.064327456187778

In [14]:
cost_range_list = []; #results will be inputted in this array
#up_name = "tri_media_filtration" # which unit process it applies to. TODO hould be user input.

#for value_change in pct_to_target1: # cycles through each value from MC range
for value_change in [0.4, 0.8]: #, 0.9]:

    # create and build model
    m = wt.watertap_setup(dynamic = False)
    m = wt.case_study_trains.get_case_study(name = 'carlsbad', flow = 4.5833, m = m)

    m.fs.tri_media_filtration.water_recovery.fix(value_change)

    # set variable to MC value
    wt.run_water_tap(m)
    results_table = get_results_table(m, unit_process_name)
    cost_range_list.append(results_table.total_up_cost.sum())


/Users/amiara/NAWI/WaterTap/NAWI-WaterTAP3/IDAES-WaterTAP3_v2/ml_regression.py:149: RuntimeWarning: divide by zero encountered in power
  return a*np.power(x, b)
/Users/amiara/NAWI/WaterTap/NAWI-WaterTAP3/IDAES-WaterTAP3_v2/ml_regression.py:149: RuntimeWarning: divide by zero encountered in power
  return a*np.power(x, b)
/Users/amiara/NAWI/WaterTap/NAWI-WaterTAP3/IDAES-WaterTAP3_v2/ml_regression.py:149: RuntimeWarning: divide by zero encountered in power
  return a*np.power(x, b)
/Users/amiara/NAWI/WaterTap/NAWI-WaterTAP3/IDAES-WaterTAP3_v2/ml_regression.py:149: RuntimeWarning: divide by zero encountered in power
  return a*np.power(x, b)
/Users/amiara/NAWI/WaterTap/NAWI-WaterTAP3/IDAES-WaterTAP3_v2/ml_regression.py:149: RuntimeWarning: divide by zero encountered in power
  return a*np.power(x, b)
/Users/amiara/NAWI/WaterTap/NAWI-WaterTAP3/IDAES-WaterTAP3_v2/ml_regression.py:149: RuntimeWarning: divide by zero encountered in power
  return a*np.power(x, b)
/Users/amiara/NAWI/WaterTap/

Ipopt 3.12.13: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.13, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      755
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      133

Total number of variables............................:      283
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        1
                     variables with only upper bounds:        0
Tot

/Users/amiara/NAWI/WaterTap/NAWI-WaterTAP3/IDAES-WaterTAP3_v2/ml_regression.py:149: RuntimeWarning: divide by zero encountered in power
  return a*np.power(x, b)
/Users/amiara/NAWI/WaterTap/NAWI-WaterTAP3/IDAES-WaterTAP3_v2/ml_regression.py:149: RuntimeWarning: divide by zero encountered in power
  return a*np.power(x, b)
/Users/amiara/NAWI/WaterTap/NAWI-WaterTAP3/IDAES-WaterTAP3_v2/ml_regression.py:149: RuntimeWarning: divide by zero encountered in power
  return a*np.power(x, b)
/Users/amiara/NAWI/WaterTap/NAWI-WaterTAP3/IDAES-WaterTAP3_v2/ml_regression.py:149: RuntimeWarning: divide by zero encountered in power
  return a*np.power(x, b)
/Users/amiara/NAWI/WaterTap/NAWI-WaterTAP3/IDAES-WaterTAP3_v2/ml_regression.py:149: RuntimeWarning: divide by zero encountered in power
  return a*np.power(x, b)
/Users/amiara/NAWI/WaterTap/NAWI-WaterTAP3/IDAES-WaterTAP3_v2/ml_regression.py:149: RuntimeWarning: divide by zero encountered in power
  return a*np.power(x, b)
/Users/amiara/NAWI/WaterTap/

Ipopt 3.12.13: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.13, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      755
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      133

Total number of variables............................:      283
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        1
                     variables with only upper bounds:        0
Tot

WARNING: Loading a SolverResults object with a warning status into
    model=unknown;
        message from solver=Ipopt 3.12.13\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
degrees_of_freedom: 0
----------------------------------------------------------------------
fs.source1
inlet : Size=1
    Key  : Name        : Value
    None :   conc_mass : {(0.0, 'TDS'): 720.0, (0.0, 'TOC'): 140.0, (0.0, 'nitrates'): 0.0}
         :    flow_vol : {0.0: 4.5833}
         :    pressure : {0.0: 200000.0}
         : temperature : {0.0: 300}
outlet : Size=1
    Key  : Name        : Value
    None :   conc_mass : {(0.0, 'TDS'): 719.9280142733767, (0.0, 'TOC'): 139.9860027753788, (0.0, 'nitrates'): -1.0042434530935294e-21}
         :    flow_vol : {0.0: 4.58329990913096}
         :    pressure : {0.0: 199999.9999}
         : temperature : {0.0: 300.0}
waste : Size=1
    Key  : Name        : Value
    None :   conc_mass : {(0.0, 'TDS'): 1676.8518595617677, (0.0, 'TOC'):

In [15]:
cost_range_list

[451.8380234584197, 461.8182604062696]

In [ ]:
import time
from multiprocessing import Pool
import multiprocessing

mu = 0.6
sigma = .1
num_reps = 50

input_list = np.random.normal(mu,sigma, size = num_reps) #, sigma, num_reps).round(4)

count, bins, ignored = plt.hist(input_list, 25, density=True)
plt.plot(bins, 1/(sigma * np.sqrt(2 * np.pi)) * np.exp( - (bins - mu)**2 / (2 * sigma**2) ),
          linewidth=2, color='r')
plt.show()

### INPUT TO MODEL LIST: ### CAN BE AUTOMATED FOR USER TO LABEL THE VARIABLE. TOOD ###
no_of_proc = 4
list_final = []
for i in range(no_of_proc):
    part2 = len(input_list) / no_of_proc
    i2 = ((i+1)*part2)
    list1 = input_list[int(i*part2):int(i2)]
    list_final.append(list1)
    
    
def monte_run(list_final):
    print('goes in')

    up_name = "tri_media_filtration" # which unit process it applies to. TODO hould be user input.
    cost_range_list = []; #results will be inputted in this array

    #for value_change in pct_to_target1: # cycles through each value from MC range
    for value_change in list_final:

        # create and build model
        m = wt.watertap_setup(dynamic = False)
        m = wt.case_study_trains.get_case_study(name = 'carlsbad', flow = 4.5833, m = m)

        getattr(m.fs, up_name).water_recovery.fix(value_change)

        # set variable to MC value
        result = wt.run_water_tap(m)
        results_table = get_results_table(m, unit_process_names)
        cost_range_list.append(results_table.total_up_cost.sum())


    return cost_range_list

startTime = time.time()

pool=Pool()
dfs = pool.map(monte_run, list_final) #SomeClass().preprocess_data()

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))

In [35]:
####TO DO LOAD AND SAVE!!

In [36]:
#### SAVE TRAIN ####
# path = 'trains/Tutorial1_treatment_train_example.csv'
# wt.save_train(T, path)

In [37]:
# #### LOAD TRAIN ####
# path = 'trains/Tutorial1_treatment_train_example.csv'
# TT = wt.load_train(path)

In [38]:
# wt.display.show_train(TT)